# **intro**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import StandardScaler

# Load the pre-trained CNN model
model = load_model('/content/drive/MyDrive/Depression/Model/finetune_on_android.h5')

# Define function to load CSV data and preprocess
def load_data(csv_path):
    df = pd.read_csv(csv_path)
    X = df.drop(['name', 'Class'], axis=1)  # Assuming 'name' and 'Class' are not features
    y = df['Class']
    # Encode class labels
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)
    # Normalize features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    return X, y

# Paths to CSV files for training and testing data of both datasets
interview_train_csv = '/content/drive/MyDrive/Depression/Android Dataset/Interview_train.csv'
interview_test_csv = '/content/drive/MyDrive/Depression/Android Dataset/Interview_test.csv'
reading_train_csv = '/content/drive/MyDrive/Depression/Android Dataset/Reading_train.csv'
reading_test_csv = '/content/drive/MyDrive/Depression/Android Dataset/Reading_test.csv'

# Load data for Interview dataset
X_interview_train, y_interview_train = load_data(interview_train_csv)
X_interview_test, y_interview_test = load_data(interview_test_csv)

# Load data for Reading dataset
X_reading_train, y_reading_train = load_data(reading_train_csv)
X_reading_test, y_reading_test = load_data(reading_test_csv)

# Compile the model (if necessary)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define callbacks (optional but recommended, e.g., for saving best model during training)
checkpoint_callback = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss', mode='min', verbose=1)

# Evaluate the model on Interview train data before fine-tuning
interview_train_accuracy_before = accuracy_score(y_interview_train, model.predict_classes(X_interview_train))
print(f'Accuracy on Interview train data before fine-tuning: {interview_train_accuracy_before}')

# Train the model on Interview dataset
model.fit(X_interview_train, y_interview_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[checkpoint_callback])

# Evaluate the model on Interview train data after fine-tuning
interview_train_accuracy_after = accuracy_score(y_interview_train, model.predict_classes(X_interview_train))
print(f'Accuracy on Interview train data after fine-tuning: {interview_train_accuracy_after}')

# Print message if catastrophic forgetting occurred
if interview_train_accuracy_after < interview_train_accuracy_before:
    print("Catastrophic forgetting occurs here.")

# Load the best model saved during training (optional, if using callbacks)
best_model = load_model('best_model.h5')

# Evaluate the model on Interview test data
interview_test_accuracy = accuracy_score(y_interview_test, best_model.predict_classes(X_interview_test))
print(f'Accuracy on Interview test data: {interview_test_accuracy}')

# Evaluate the model on Reading test data
reading_test_accuracy = accuracy_score(y_reading_test, best_model.predict_classes(X_reading_test))
print(f'Accuracy on Reading test data: {reading_test_accuracy}')


AttributeError: 'Functional' object has no attribute 'predict_classes'

In [ ]:
import pandas as pd

def csv_details(file_path):
    # Load CSV file into a DataFrame
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print("File not found.")
        return

    # Display basic details
    print("Columns:")
    print(df.columns.tolist())
    print("\nData Types:")
    print(df.dtypes)
    print("\nPreview (first 5 rows):")
    print(df.head())

# Example usage:
csv_file_path = '/content/drive/MyDrive/Depression/Android Dataset/Interview_train.csv'  # Replace this with the path to your CSV file
csv_details(csv_file_path)


Columns:
['name', 'Class', 'feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_15', 'feature_16', 'feature_17', 'feature_18', 'feature_19', 'feature_20', 'feature_21', 'feature_22', 'feature_23', 'feature_24', 'feature_25', 'feature_26', 'feature_27', 'feature_28', 'feature_29', 'feature_30', 'feature_31', 'feature_32', 'feature_33', 'feature_34', 'feature_35', 'feature_36', 'feature_37', 'feature_38', 'feature_39', 'feature_40', 'feature_41', 'feature_42', 'feature_43', 'feature_44', 'feature_45', 'feature_46', 'feature_47', 'feature_48', 'feature_49', 'feature_50', 'feature_51', 'feature_52', 'feature_53', 'feature_54', 'feature_55', 'feature_56', 'feature_57', 'feature_58', 'feature_59', 'feature_60', 'feature_61', 'feature_62', 'feature_63', 'feature_64', 'feature_65', 'feature_66', 'feature_67', 'feature_68', 'feature_69', 'fe

understanding the blog code


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

(mnist_train, mnist_test), ds_info = tfds.load('mnist', split=['train', 'test'], as_supervised=True, with_info=True)

def normalize_img(image, label):
  return tf.cast(image, tf.float32) / 255., label

def transform_labels(image, label):
  return image, tf.math.floor(label / 2)

def prepare(ds, shuffle=True, batch_size=32, prefetch=True):
  ds = ds.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
  ds = ds.map(transform_labels, num_parallel_calls=tf.data.experimental.AUTOTUNE)
  ds = ds.shuffle(ds_info.splits['train'].num_examples) if shuffle else ds
  ds = ds.cache()
  ds = ds.batch(batch_size)
  ds = ds.prefetch(tf.data.experimental.AUTOTUNE) if prefetch else ds
  return ds

def split_tasks(ds, predicate):
  return ds.filter(predicate), ds.filter(lambda img, label: not predicate(img, label))

multi_task_train, multi_task_test = prepare(mnist_train), prepare(mnist_test)
task_A_train, task_B_train = split_tasks(mnist_train, lambda img, label: label % 2 == 0)
task_A_train, task_B_train = prepare(task_A_train), prepare(task_B_train)
task_A_test, task_B_test = split_tasks(mnist_test, lambda img, label: label % 2 == 0)
task_A_test, task_B_test = prepare(task_A_test), prepare(task_B_test)

def evaluate(model, test_set):
  acc = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')
  for i, (imgs, labels) in enumerate(test_set):
    preds = model.predict_on_batch(imgs)
    acc.update_state(labels, preds)
  return acc.result().numpy()

multi_task_model = tf.keras.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(5)
])

multi_task_model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics='accuracy')

multi_task_model.fit(multi_task_train, epochs=6)

print("Task A accuracy after training on Multi-Task Problem: {}".format(evaluate(multi_task_model, task_A_test)))
print("Task B accuracy after training on Multi-Task Problem: {}".format(evaluate(multi_task_model, task_B_test)))


Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.
Epoch 1/6
1875/1875 [==============================] - 16s 5ms/step - loss: 0.2257 - accuracy: 0.9306
Epoch 2/6
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1027 - accuracy: 0.9688
Epoch 3/6
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0728 - accuracy: 0.9774
Epoch 4/6
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0543 - accuracy: 0.9839
Epoch 5/6
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0413 - accuracy: 0.9881
Epoch 6/6
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0313 - accuracy: 0.9916


Task A accuracy after training on Multi-Task Problem: 0.9803085923194885
Task B accuracy after training on Multi-Task Problem: 0.9716200232505798


In [ ]:
basic_cl_model = tf.keras.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(5)
])

basic_cl_model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics='accuracy')

basic_cl_model.fit(task_A_train, epochs=6)

print("Task A accuracy after training model on only Task A: {}".format(evaluate(basic_cl_model, task_A_test)))

basic_cl_model.fit(task_B_train, epochs=6)

print("Task B accuracy after training trained model on Task B: {}".format(evaluate(basic_cl_model, task_B_test)))
print("Task A accuracy after training trained model on Task B: {}".format(evaluate(basic_cl_model, task_A_test)))


Epoch 1/6
922/922 [==============================] - 6s 3ms/step - loss: 0.1384 - accuracy: 0.9592
Epoch 2/6
922/922 [==============================] - 3s 4ms/step - loss: 0.0569 - accuracy: 0.9829
Epoch 3/6
922/922 [==============================] - 3s 3ms/step - loss: 0.0358 - accuracy: 0.9895
Epoch 4/6
922/922 [==============================] - 4s 5ms/step - loss: 0.0238 - accuracy: 0.9935
Epoch 5/6
922/922 [==============================] - 3s 3ms/step - loss: 0.0166 - accuracy: 0.9963
Epoch 6/6
922/922 [==============================] - 3s 3ms/step - loss: 0.0111 - accuracy: 0.9974
Task A accuracy after training model on only Task A: 0.986804723739624
Epoch 1/6
954/954 [==============================] - 7s 4ms/step - loss: 0.1935 - accuracy: 0.9487
Epoch 2/6
954/954 [==============================] - 4s 4ms/step - loss: 0.0641 - accuracy: 0.9809
Epoch 3/6
954/954 [==============================] - 4s 4ms/step - loss: 0.0427 - accuracy: 0.9880
Epoch 4/6
954/954 [===================

In [ ]:
def l2_penalty(theta, theta_A):
  penalty = 0
  for i, theta_i in enumerate(theta):
    _penalty = tf.math.reduce_sum((theta_i - theta_A[i]) ** 2)
    penalty += _penalty
  return 0.5*penalty

def train_with_l2(model, task_A_train, task_B_train, task_A_test, task_B_test, epochs=6):
  model.fit(task_A_train, epochs=epochs)
  theta_A = {n: p.value() for n, p in enumerate(model.trainable_variables.copy())}

  print("Task A accuracy after training on Task A: {}".format(evaluate(model, task_A_test)))
  # Now we set up the training loop for task B with EWC
  accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
  loss = tf.keras.metrics.SparseCategoricalCrossentropy('loss')

  for epoch in range(epochs*3):
    accuracy.reset_states()
    loss.reset_states()

    for batch, (imgs, labels) in enumerate(task_B_set):
      with tf.GradientTape() as tape:
        # Make the predictions
        preds = model(imgs)
        # Compute EWC loss
        total_loss = ewc_loss(labels, preds, model, F, theta_A)
      # Compute the gradients of model's trainable parameters wrt total loss
      grads = tape.gradient(total_loss, model.trainable_variables)
      # Update the model with gradients
      model.optimizer.apply_gradients(zip(grads, model.trainable_variables))
      # Report updated loss and accuracy
      accuracy.update_state(labels, preds)
      loss.update_state(labels, preds)
      print("\rEpoch: {}, Batch: {}, Loss: {:.3f}, Accuracy: {:.3f}".format(
          epoch+1, batch+1, loss.result().numpy(), accuracy.result().numpy()), flush=True, end=''
         )
    print("")

  print("Task B accuracy after training trained model on Task B: {}".format(evaluate(model, task_B_test)))
  print("Task A accuracy after training trained model on Task B: {}".format(evaluate(model, task_A_test)))

# Define EWC model
ewc_model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(5)
])

In [ ]:
l2_model = tf.keras.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(5)
])

l2_model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics='accuracy')

train_with_l2(l2_model, task_A_train, task_B_train, task_A_test, task_B_test)

def compute_precision_matrices(model, task_set, num_batches=1, batch_size=32):
  task_set = task_set.repeat()
  precision_matrices = {n: tf.zeros_like(p.value()) for n, p in enumerate(model.trainable_variables)}

  for i, (imgs, labels) in enumerate(task_set.take(num_batches)):
    with tf.GradientTape() as tape:
      preds = model(imgs)
      ll = tf.nn.log_softmax(preds)
    ll_grads = tape.gradient(ll, model.trainable_variables)
    for i, g in enumerate(ll_grads):
      precision_matrices[i] += tf.math.reduce_mean(g ** 2, axis=0) / num_batches

  return precision_matrices

def compute_elastic_penalty(F, theta, theta_A, alpha=25):
  penalty = 0
  for i, theta_i in enumerate(theta):
    _penalty = tf.math.reduce_sum(F[i] * (theta_i - theta_A[i]) ** 2)
    penalty += _penalty
  return 0.5*alpha*penalty

def ewc_loss(labels, preds, model, F, theta_A):
  loss_b = model.loss(labels, preds)
  penalty = compute_elastic_penalty(F, model.trainable_variables, theta_A)
  return loss_b + penalty

def train_with_ewc(model, task_A_set, task_B_set, task_A_test, task_B_test, epochs=3):
  # First we're going to fit to task A and retain a copy of parameters trained on Task A
  model.fit(task_A_set, epochs=epochs)
  theta_A = {n: p.value() for n, p in enumerate(model.trainable_variables.copy())}
  # We'll only compute Fisher once, you can do it whenever
  F = compute_precision_matrices(model, task_A_set, num_batches=1000)

  print("Task A accuracy after training on Task A: {}".format(evaluate(model, task_A_test)))


Epoch 1/6
922/922 [==============================] - 5s 5ms/step - loss: 0.1333 - accuracy: 0.9614
Epoch 2/6
922/922 [==============================] - 3s 3ms/step - loss: 0.0556 - accuracy: 0.9836
Epoch 3/6
922/922 [==============================] - 3s 3ms/step - loss: 0.0357 - accuracy: 0.9891
Epoch 4/6
922/922 [==============================] - 4s 4ms/step - loss: 0.0240 - accuracy: 0.9936
Epoch 5/6
922/922 [==============================] - 3s 3ms/step - loss: 0.0159 - accuracy: 0.9963
Epoch 6/6
922/922 [==============================] - 4s 5ms/step - loss: 0.0110 - accuracy: 0.9978
Task A accuracy after training on Task A: 0.9839626550674438


NameError: name 'task_B_set' is not defined

In [ ]:
# Now we set up the training loop for task B with EWC
  accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
  loss = tf.keras.metrics.SparseCategoricalCrossentropy('loss')

  for epoch in range(epochs*3):
    accuracy.reset_states()
    loss.reset_states()

    for batch, (imgs, labels) in enumerate(task_B_set):
      with tf.GradientTape() as tape:
        # Make the predictions
        preds = model(imgs)
        # Compute EWC loss
        total_loss = ewc_loss(labels, preds, model, F, theta_A)
      # Compute the gradients of model's trainable parameters wrt total loss
      grads = tape.gradient(total_loss, model.trainable_variables)
      # Update the model with gradients
      model.optimizer.apply_gradients(zip(grads, model.trainable_variables))
      # Report updated loss and accuracy
      accuracy.update_state(labels, preds)
      loss.update_state(labels, preds)
      print("\rEpoch: {}, Batch: {}, Loss: {:.3f}, Accuracy: {:.3f}".format(
          epoch+1, batch+1, loss.result().numpy(), accuracy.result().numpy()), flush=True, end=''
         )
    print("")

  print("Task B accuracy after training trained model on Task B: {}".format(evaluate(model, task_B_test)))
  print("Task A accuracy after training trained model on Task B: {}".format(evaluate(model, task_A_test)))

# Define EWC model
ewc_model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(5)
])

# **1st step of code implementation**

In [ ]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/Depression/Android Dataset/Reading_test.csv')
df.head()

,name,Class,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_502,feature_503,feature_504,feature_505,feature_506,feature_507,feature_508,feature_509,feature_510,feature_511
0,Reading_depressed_0.wav,1,-23.453386,-6.972407,10.922126,8.391150,-0.868982,-18.249210,-12.701187,-4.695340,...,-20.443972,7.936236,13.323593,-4.330662,-7.336786,14.697271,14.230615,15.683101,16.694450,-17.903179
1,Reading_depressed_1.wav,1,-25.416334,-9.199664,10.101542,10.495510,3.363707,-7.523287,-24.693653,-5.431184,...,-25.981260,10.707705,17.087761,-3.575731,-4.642745,10.044394,7.741048,16.284454,16.271326,-14.527636
2,Reading_depressed_10.wav,1,-28.969011,-5.609468,12.488464,16.134411,15.011572,-12.213967,-19.554272,-4.658757,...,-24.819601,13.386227,-3.435347,-4.213245,-17.756733,12.055525,12.868338,7.721011,14.429776,-17.652357
3,Reading_depressed_100.wav,1,-19.701765,12.261052,7.070023,13.615430,11.327873,1.511992,-23.616589,9.273006,...,-22.441940,9.820329,-11.617693,-4.013164,-9.241024,8.637423,8.017566,-0.484947,13.701614,-9.746015
4,Reading_depressed_101.wav,1,-22.105116,-7.490783,10.399014,10.492347,-1.390268,-12.003224,-16.849686,-1.143140,...,-19.453552,13.122263,17.303165,-3.531888,-10.645234,12.287755,5.714491,13.738103,10.087270,-14.942650


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np

# Load and process the first audio dataset
audio_train_path_1 = "/content/drive/MyDrive/Depression/EDAIC Dataset/training_labels.csv"
audio_test_path_1 = "/content/drive/MyDrive/Depression/EDAIC Dataset/testing_labels.csv"

audio_train_df_1 = pd.read_csv(audio_train_path_1)
audio_test_df_1 = pd.read_csv(audio_test_path_1)

# Load and process the second audio dataset
audio_train_path_2 = "/content/drive/MyDrive/Depression/Android Dataset/Reading_train.csv"
audio_test_path_2 = "/content/drive/MyDrive/Depression/Android Dataset/Reading_test.csv"

audio_train_df_2 = pd.read_csv(audio_train_path_2)
audio_test_df_2 = pd.read_csv(audio_test_path_2)

# Randomly sample rows for training and testing from the first dataset
audio_train_df_1 = audio_train_df_1.sample(frac=1).reset_index(drop=True)
audio_test_df_1 = audio_test_df_1.sample(frac=1).reset_index(drop=True)

# Randomly sample rows for training and testing from the second dataset
audio_train_df_2 = audio_train_df_2.sample(frac=1).reset_index(drop=True)
audio_test_df_2 = audio_test_df_2.sample(frac=1).reset_index(drop=True)

# Convert audio features to numpy arrays for the first dataset
X_train_1 = audio_train_df_1.drop(columns=['name']).values.astype(np.float32)
y_train_1 = audio_train_df_1['Class'].values.astype(np.int32)  # Convert labels to integers
X_test_1 = audio_test_df_1.drop(columns=['name']).values.astype(np.float32)
y_test_1 = audio_test_df_1['Class'].values.astype(np.int32)  # Convert labels to integers

# Convert audio features to numpy arrays for the second dataset
X_train_2 = audio_train_df_2.drop(columns=['name']).values.astype(np.float32)
y_train_2 = audio_train_df_2['Class'].values.astype(np.int32)  # Convert labels to integers
X_test_2 = audio_test_df_2.drop(columns=['name']).values.astype(np.float32)
y_test_2 = audio_test_df_2['Class'].values.astype(np.int32)  # Convert labels to integers

# Convert audio datasets to TensorFlow datasets for the first dataset
audio_train_ds_1 = tf.data.Dataset.from_tensor_slices((X_train_1, y_train_1))
audio_test_ds_1 = tf.data.Dataset.from_tensor_slices((X_test_1, y_test_1))

# Convert audio datasets to TensorFlow datasets for the second dataset
audio_train_ds_2 = tf.data.Dataset.from_tensor_slices((X_train_2, y_train_2))
audio_test_ds_2 = tf.data.Dataset.from_tensor_slices((X_test_2, y_test_2))

# Shuffle and batch the datasets for the first dataset
BATCH_SIZE = 32
SHUFFLE_SIZE = 1000
audio_train_ds_1 = audio_train_ds_1.shuffle(SHUFFLE_SIZE).batch(BATCH_SIZE)
audio_test_ds_1 = audio_test_ds_1.batch(BATCH_SIZE)

# Shuffle and batch the datasets for the second dataset
audio_train_ds_2 = audio_train_ds_2.shuffle(SHUFFLE_SIZE).batch(BATCH_SIZE)
audio_test_ds_2 = audio_test_ds_2.batch(BATCH_SIZE)

# Load your pre-trained CNN model
model_path = '/content/drive/MyDrive/Depression/Model/finetune_on_latest.h5'
cnn_model = tf.keras.models.load_model(model_path)
print(cnn_model.summary())

# Fine-tune your CNN model on the first dataset
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
cnn_model.fit(audio_train_ds_1, epochs=10)

# Evaluate your CNN model on the first dataset
test_loss_1, test_accuracy_1 = cnn_model.evaluate(audio_test_ds_1)
print("Test Loss (Dataset 1):", test_loss_1)
print("Test Accuracy (Dataset 1):", test_accuracy_1)

# Fine-tune your CNN model on the second dataset
cnn_model.fit(audio_train_ds_2, epochs=10)

# Evaluate your CNN model on the second dataset
test_loss_2, test_accuracy_2 = cnn_model.evaluate(audio_test_ds_2)
print("Test Loss (Dataset 2):", test_loss_2)
print("Test Accuracy (Dataset 2):", test_accuracy_2)


<ipython-input-7-dc06bdd41489>:29: RuntimeWarning: invalid value encountered in cast
  y_train_1 = audio_train_df_1['Class'].values.astype(np.int32)  # Convert labels to integers


Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 512, 1)]          0         
                                                                 
 conv1d_9 (Conv1D)           (None, 510, 32)           128       
                                                                 
 max_pooling1d_9 (MaxPoolin  (None, 255, 32)           0         
 g1D)                                                            
                                                                 
 conv1d_10 (Conv1D)          (None, 253, 64)           6208      
                                                                 
 max_pooling1d_10 (MaxPooli  (None, 126, 64)           0         
 ng1D)                                                           
                                                                 
 flatten_6 (Flatten)         (None, 8064)              0   

ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueError: In

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np

# Load and process the first audio dataset
audio_train_path_1 = "/content/drive/MyDrive/Depression/Android Dataset/Interview_train.csv"
audio_test_path_1 = "/content/drive/MyDrive/Depression/Android Dataset/Interview_test.csv"

audio_train_df_1 = pd.read_csv(audio_train_path_1)
audio_test_df_1 = pd.read_csv(audio_test_path_1)

# Load and process the second audio dataset
audio_train_path_2 = "/content/drive/MyDrive/Depression/Android Dataset/Reading_train.csv"
audio_test_path_2 = "/content/drive/MyDrive/Depression/Android Dataset/Reading_test.csv"

audio_train_df_2 = pd.read_csv(audio_train_path_2)
audio_test_df_2 = pd.read_csv(audio_test_path_2)

# Convert audio features to numpy arrays for the first dataset
X_train_1 = audio_train_df_1.drop(columns=['name']).values.astype(np.float32)
y_train_1 = audio_train_df_1['Class'].values.astype(np.float32)
X_test_1 = audio_test_df_1.drop(columns=['name']).values.astype(np.float32)
y_test_1 = audio_test_df_1['Class'].values.astype(np.float32)

# Convert audio features to numpy arrays for the second dataset
X_train_2 = audio_train_df_2.drop(columns=['name']).values.astype(np.float32)
y_train_2 = audio_train_df_2['Class'].values.astype(np.float32)
X_test_2 = audio_test_df_2.drop(columns=['name']).values.astype(np.float32)
y_test_2 = audio_test_df_2['Class'].values.astype(np.float32)

# Reshape audio features for CNN input (assuming 1D convolution)
X_train_1 = X_train_1.reshape(-1, X_train_1.shape[1], 1)
X_test_1 = X_test_1.reshape(-1, X_test_1.shape[1], 1)
X_train_2 = X_train_2.reshape(-1, X_train_2.shape[1], 1)
X_test_2 = X_test_2.reshape(-1, X_test_2.shape[1], 1)

# Convert audio datasets to TensorFlow datasets for the first dataset
audio_train_ds_1 = tf.data.Dataset.from_tensor_slices((X_train_1, y_train_1))
audio_test_ds_1 = tf.data.Dataset.from_tensor_slices((X_test_1, y_test_1))

# Convert audio datasets to TensorFlow datasets for the second dataset
audio_train_ds_2 = tf.data.Dataset.from_tensor_slices((X_train_2, y_train_2))
audio_test_ds_2 = tf.data.Dataset.from_tensor_slices((X_test_2, y_test_2))

# Shuffle and batch the datasets for the first dataset
BATCH_SIZE = 32
SHUFFLE_SIZE = 1000
audio_train_ds_1 = audio_train_ds_1.shuffle(SHUFFLE_SIZE).batch(BATCH_SIZE)
audio_test_ds_1 = audio_test_ds_1.batch(BATCH_SIZE)

# Shuffle and batch the datasets for the second dataset
audio_train_ds_2 = audio_train_ds_2.shuffle(SHUFFLE_SIZE).batch(BATCH_SIZE)
audio_test_ds_2 = audio_test_ds_2.batch(BATCH_SIZE)

# Load your model
model_path = '/content/drive/MyDrive/Depression/Model/finetune_on_latest.h5'
cnn_model = tf.keras.models.load_model(model_path)

# Train your CNN model on the first dataset
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
cnn_model.fit(audio_train_ds_1, epochs=10)

# Evaluate your model on the first dataset
test_loss_1, test_accuracy_1 = cnn_model.evaluate(audio_test_ds_1)
print("Test Loss (Dataset 1):", test_loss_1)
print("Test Accuracy (Dataset 1):", test_accuracy_1)

# Train your CNN model on the second dataset
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
cnn_model.fit(audio_train_ds_2, epochs=10)

# Evaluate your model on the second dataset
test_loss_2, test_accuracy_2 = cnn_model.evaluate(audio_test_ds_2)
print("Test Loss (Dataset 2):", test_loss_2)
print("Test Accuracy (Dataset 2):", test_accuracy_2)


Epoch 1/10


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueError: In

# **Final Codes**

# **Catestrophic Forgetting**

# Loading And Testing CNN model on EDAIC

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.models import load_model

# Load the saved model

model_save_path = '/content/drive/MyDrive/Depression/Model/latest_CNN.h5'  # Update with your model's path
loaded_model = load_model(model_save_path)
print("Model loaded successfully.")

# Load and prepare the test data
test_data_path = '/content/drive/MyDrive/Depression/EDAIC Dataset/testing_labels.csv'  # Update with your test data's path
test_data = pd.read_csv(test_data_path)

# Assuming your CSV has specific columns to drop (like 'name') and a 'Class' column for labels
X_test = test_data.drop(columns=['Class', 'name']).values
y_test = test_data['Class'].values

# Scale the features
scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(X_test)

# Reshape the data if your model expects a specific input shape (example for CNN)
X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

# Make predictions on the test data
y_pred = (loaded_model.predict(X_test_scaled) > 0.5).astype(int)

# Calculate and print the classification report and accuracy
print("CNN Model Classification Report for Testing Set:")
print(classification_report(y_test, y_pred))
accuracy_test = accuracy_score(y_test, y_pred)
print(f"\nAccuracy for CNN Testing Set: {accuracy_test:.4f}")


Model loaded successfully.
77/77 [==============================] - 1s 9ms/step
CNN Model Classification Report for Testing Set:
              precision    recall  f1-score   support

           0       0.78      0.95      0.86      1897
           1       0.26      0.06      0.10       549

    accuracy                           0.75      2446
   macro avg       0.52      0.51      0.48      2446
weighted avg       0.66      0.75      0.69      2446


Accuracy for CNN Testing Set: 0.7522


# Testing on Interview Dataset

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.models import load_model

# Load the saved model
model_save_path = '/content/drive/MyDrive/Depression/Model/latest_CNN.h5'  # Update with your model's path
loaded_model = load_model(model_save_path)
print("Model loaded successfully.")

# Load and prepare the test data
test_data_path = '/content/drive/MyDrive/Depression/Android Dataset/Interview_test.csv'  # Update with your test data's path
test_data = pd.read_csv(test_data_path)

# Assuming your CSV has specific columns to drop (like 'name') and a 'Class' column for labels
X_test = test_data.drop(columns=['Class', 'name']).values
y_test = test_data['Class'].values

# Scale the features
scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(X_test)

# Reshape the data if your model expects a specific input shape (example for CNN)
X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

# Make predictions on the test data
y_pred = (loaded_model.predict(X_test_scaled) > 0.5).astype(int)

# Calculate and print the classification report and accuracy
print("CNN Model Classification Report for Testing Set:")
print(classification_report(y_test, y_pred))
accuracy_test = accuracy_score(y_test, y_pred)
print(f"\nAccuracy for CNN Testing Set: {accuracy_test:.4f}")


Model loaded successfully.
47/47 [==============================] - 1s 23ms/step
CNN Model Classification Report for Testing Set:
              precision    recall  f1-score   support

           0       0.42      0.94      0.58       624
           1       0.62      0.07      0.13       875

    accuracy                           0.43      1499
   macro avg       0.52      0.51      0.35      1499
weighted avg       0.54      0.43      0.31      1499


Accuracy for CNN Testing Set: 0.4323


# Load and Fine-tune on Interview

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Load the training and validation dataset
interview_df = pd.read_csv('/content/drive/MyDrive/Depression/Android Dataset/Interview_train.csv')

# Prepare the dataset by dropping the 'name' column and separating features and labels
X = interview_df.drop(columns=['name', 'Class']).values
y = interview_df['Class'].values

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Load the pre-trained model
model = load_model('/content/drive/MyDrive/Depression/Model/latest_CNN.h5')

# Compile the model with a smaller learning rate
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Use early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Fine-tune the model on the new dataset
model.fit(X_train_scaled, y_train, validation_data=(X_val_scaled, y_val), epochs=10, callbacks=[early_stopping])

# Save the fine-tuned model
model.save('/content/drive/MyDrive/Depression/Model/finetune_on_latest.h5')


Epoch 1/10
85/85 [==============================] - 6s 53ms/step - loss: 1.7632 - accuracy: 0.5564 - val_loss: 0.8433 - val_accuracy: 0.5742
Epoch 2/10
85/85 [==============================] - 3s 37ms/step - loss: 0.7691 - accuracy: 0.5405 - val_loss: 0.6793 - val_accuracy: 0.6024
Epoch 3/10
85/85 [==============================] - 3s 35ms/step - loss: 0.6827 - accuracy: 0.5750 - val_loss: 0.6384 - val_accuracy: 0.6320
Epoch 4/10
85/85 [==============================] - 4s 42ms/step - loss: 0.6486 - accuracy: 0.6162 - val_loss: 0.5954 - val_accuracy: 0.7136
Epoch 5/10
85/85 [==============================] - 6s 65ms/step - loss: 0.6033 - accuracy: 0.6841 - val_loss: 0.5627 - val_accuracy: 0.7537
Epoch 6/10
85/85 [==============================] - 6s 68ms/step - loss: 0.5698 - accuracy: 0.7131 - val_loss: 0.5310 - val_accuracy: 0.7774
Epoch 7/10
85/85 [==============================] - 6s 75ms/step - loss: 0.5425 - accuracy: 0.7283 - val_loss: 0.5048 - val_accuracy: 0.7893
Epoch 8/10
85

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Test Fine-Tuned Model On Interview

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score

# Load the test dataset
test_df = pd.read_csv('/content/drive/MyDrive/Depression/Android Dataset/Interview_test.csv')

# Prepare the test dataset by dropping the 'name' column and separating features and labels
X_test = test_df.drop(columns=['name', 'Class']).values
y_test = test_df['Class'].values

# Scale the features of the test set using the same scaler as used for training
scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(X_test)

# Load the fine-tuned model
fine_tuned_model = load_model('/content/drive/MyDrive/Depression/Model/finetune_on_latest.h5')

# Make predictions on the test set
predictions = fine_tuned_model.predict(X_test_scaled)
# Convert probabilities to class labels based on a threshold (0.5 for binary classification)
predicted_classes = (predictions > 0.5).astype(int)

# Calculate and print the accuracy on the test set
test_accuracy = accuracy_score(y_test, predicted_classes)
print(f'Test Accuracy: {test_accuracy}')

47/47 [==============================] - 0s 9ms/step
Test Accuracy: 0.6504336224149433


# Test Fine-Tuned Model On EDAIC

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score

# Load the test dataset
test_df = pd.read_csv('/content/drive/MyDrive/Depression/EDAIC Dataset/testing_labels.csv')

# Prepare the test dataset by dropping the 'name' column and separating features and labels
X_test = test_df.drop(columns=['name', 'Class']).values
y_test = test_df['Class'].values

# Scale the features of the test set using the same scaler as used for training
scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(X_test)

# Load the fine-tuned model
fine_tuned_model = load_model('/content/drive/MyDrive/Depression/Model/finetune_on_latest.h5')

# Make predictions on the test set
predictions = fine_tuned_model.predict(X_test_scaled)
# Convert probabilities to class labels based on a threshold (0.5 for binary classification)
predicted_classes = (predictions > 0.5).astype(int)

# Calculate and print the accuracy on the test set
test_accuracy = accuracy_score(y_test, predicted_classes)
print(f'Test Accuracy: {test_accuracy}')

77/77 [==============================] - 1s 9ms/step
Test Accuracy: 0.669255928045789


# **next**

In [ ]:
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load and process the first audio dataset
audio_train_path_1 = "/content/drive/MyDrive/Depression/Android Dataset/Interview_train.csv"  # Update with the path to your first audio train CSV file
audio_test_path_1 = "/content/drive/MyDrive/Depression/Android Dataset/Interview_test.csv"  # Update with the path to your first audio test CSV file

audio_train_df_1 = pd.read_csv(audio_train_path_1)
audio_test_df_1 = pd.read_csv(audio_test_path_1)

# Load and process the second audio dataset
audio_train_path_2 = "/content/drive/MyDrive/Depression/Android Dataset/Reading_train.csv"  # Update with the path to your second audio train CSV file
audio_test_path_2 = "/content/drive/MyDrive/Depression/Android Dataset/Reading_test.csv"  # Update with the path to your second audio test CSV file

audio_train_df_2 = pd.read_csv(audio_train_path_2)
audio_test_df_2 = pd.read_csv(audio_test_path_2)

# Define normalization function for audio features
def normalize_audio(audio_features, label):
    scaler = MinMaxScaler()
    normalized_audio_features = scaler.fit_transform(audio_features)
    return normalized_audio_features, label

# Define function to split tasks for audio datasets
def split_audio_tasks(audio_ds, predicate):
    return audio_ds.filter(predicate), audio_ds.filter(lambda audio, label: not predicate(audio, label))

# Prepare audio datasets
def prepare_audio(ds, shuffle=True, batch_size=32, prefetch=True):
    ds = ds.map(normalize_audio, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    if shuffle:
        ds = ds.shuffle(SHUFFLE_SIZE)
    ds = ds.cache()
    ds = ds.batch(batch_size)
    if prefetch:
        ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds

# Define shuffle size
SHUFFLE_SIZE = 1000  # Update with appropriate value

# Convert audio datasets to TensorFlow datasets
audio_train_ds_1 = tf.data.Dataset.from_tensor_slices((audio_train_df_1.drop(columns=['name']).values, audio_train_df_1['Class'].values))
audio_test_ds_1 = tf.data.Dataset.from_tensor_slices((audio_test_df_1.drop(columns=['name']).values, audio_test_df_1['Class'].values))

audio_train_ds_2 = tf.data.Dataset.from_tensor_slices((audio_train_df_2.drop(columns=['name']).values, audio_train_df_2['Class'].values))
audio_test_ds_2 = tf.data.Dataset.from_tensor_slices((audio_test_df_2.drop(columns=['name']).values, audio_test_df_2['Class'].values))

# Split tasks for audio datasets if needed
# task_A_train, task_B_train = split_audio_tasks(audio_train_ds, YOUR_PREDICATE_FUNCTION)
# task_A_test, task_B_test = split_audio_tasks(audio_test_ds, YOUR_PREDICATE_FUNCTION)

# Prepare audio datasets
# task_A_train = prepare_audio(task_A_train)
# task_B_train = prepare_audio(task_B_train)
# task_A_test = prepare_audio(task_A_test)
# task_B_test = prepare_audio(task_B_test)

# Load your CNN model
cnn_model = tf.keras.models.load_model('/content/drive/MyDrive/Depression/Model/finetune_on_android.h5')

# Define evaluation function for audio data
def evaluate_audio(model, test_set):
    accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')
    for audio_features, labels in test_set:
        preds = model.predict(audio_features)
        accuracy.update_state(labels, preds)
    return accuracy.result().numpy()

# Define training function for audio data
def train_audio_model(model, train_set, epochs):
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(train_set, epochs=10)

# Define number of epochs
EPOCHS = 10  # Update with appropriate value

# Train your CNN model on task A
train_audio_model(cnn_model, audio_train_ds_1, EPOCHS)

# Evaluate on task A
task_A_accuracy_1 = evaluate_audio(cnn_model, audio_test_ds_1)
print("Task A accuracy for dataset 1: {}".format(task_A_accuracy_1))

# Train your CNN model on task B
train_audio_model(cnn_model, audio_train_ds_2, EPOCHS)

# Evaluate on task B
task_B_accuracy_2 = evaluate_audio(cnn_model, audio_test_ds_2)
print("Task B accuracy for dataset 2: {}".format(task_B_accuracy_2))


Epoch 1/10


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/input_spec.py", line 235, in assert_input_compatibility
        raise ValueError(

    ValueError: Ex

In [ ]:
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load and process the audio dataset
audio_train_path = "/content/drive/MyDrive/Depression/Android Dataset/Interview_train.csv"  # Update with the path to your audio train CSV file
audio_test_path = "/content/drive/MyDrive/Depression/Android Dataset/Interview_test.csv"  # Update with the path to your audio test CSV file

audio_train_df = pd.read_csv(audio_train_path)
audio_test_df = pd.read_csv(audio_test_path)

# Define normalization function for audio features
def normalize_audio(audio_features, label):
    scaler = MinMaxScaler()
    # Convert TensorFlow tensor to NumPy array
    audio_features_np = audio_features.numpy()
    normalized_audio_features = scaler.fit_transform(audio_features_np)
    return normalized_audio_features, label


# Convert audio datasets to TensorFlow datasets
audio_train_ds = tf.data.Dataset.from_tensor_slices((audio_train_df.drop(columns=['name']).values, audio_train_df['Class'].values))
audio_test_ds = tf.data.Dataset.from_tensor_slices((audio_test_df.drop(columns=['name']).values, audio_test_df['Class'].values))

# Prepare audio datasets
def prepare_audio(ds, shuffle=True, batch_size=32, prefetch=True):
    ds = ds.map(normalize_audio, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    if shuffle:
        ds = ds.shuffle(SHUFFLE_SIZE)
    ds = ds.cache()
    ds = ds.batch(batch_size)
    if prefetch:
        ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds

# Define shuffle size
SHUFFLE_SIZE = 1000  # Update with appropriate value

# Prepare audio datasets
audio_train_ds = prepare_audio(audio_train_ds)
audio_test_ds = prepare_audio(audio_test_ds)

# Load your model
model_path = '/content/drive/MyDrive/Depression/Model/finetune_on_latest.h5'
cnn_model = tf.keras.models.load_model(model_path)

# Train your CNN model
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
cnn_model.fit(audio_train_ds, epochs=10)

# Evaluate your model
test_loss, test_accuracy = cnn_model.evaluate(audio_test_ds)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


AttributeError: in user code:

    File "<ipython-input-3-7c685e61f090>", line 16, in normalize_audio  *
        audio_features_np = audio_features.numpy()

    AttributeError: 'SymbolicTensor' object has no attribute 'numpy'


# model details

In [ ]:
import tensorflow as tf

# Load the model
model_path = '/content/drive/MyDrive/Depression/Model/finetune_on_latest.h5'
loaded_model = tf.keras.models.load_model(model_path)

# Print the model summary
print(loaded_model.summary())


Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 512, 1)]          0         
                                                                 
 conv1d_9 (Conv1D)           (None, 510, 32)           128       
                                                                 
 max_pooling1d_9 (MaxPoolin  (None, 255, 32)           0         
 g1D)                                                            
                                                                 
 conv1d_10 (Conv1D)          (None, 253, 64)           6208      
                                                                 
 max_pooling1d_10 (MaxPooli  (None, 126, 64)           0         
 ng1D)                                                           
                                                                 
 flatten_6 (Flatten)         (None, 8064)              0   

In [ ]:
import tensorflow as tf

def get_model_details(model_path):
    # Load the model
    model = tf.keras.models.load_model(model_path)

    # Print model architecture
    print("Model Architecture:")
    print(model.summary())

    # Print input shape
    print("\nInput Shape:")
    print(model.input_shape)

    # Print details of each layer
    print("\nLayer Details:")
    for layer in model.layers:
        print(layer.name, "-", layer.__class__.__name__)

        # If it's a convolutional layer, print kernel size and number of filters
        if isinstance(layer, tf.keras.layers.Conv1D):
            print("  Kernel Size:", layer.kernel_size)
            print("  Filters:", layer.filters)

        # If it's a recurrent layer, print units
        elif isinstance(layer, tf.keras.layers.RNN) or isinstance(layer, tf.keras.layers.LSTM) or isinstance(layer, tf.keras.layers.GRU):
            print("  Units:", layer.units)

        # If it's a dense layer, print units
        elif isinstance(layer, tf.keras.layers.Dense):
            print("  Units:", layer.units)

        # If it's a dropout layer, print dropout rate
        elif isinstance(layer, tf.keras.layers.Dropout):
            print("  Rate:", layer.rate)

        # If it's any other layer, print its class name
        else:
            print("  Class:", layer.__class__.__name__)

        print()

# Input your model path here
model_path = '/content/drive/MyDrive/Depression/Model/finetune_on_latest.h5'

# Get model details
get_model_details(model_path)


Model Architecture:
Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 512, 1)]          0         
                                                                 
 conv1d_9 (Conv1D)           (None, 510, 32)           128       
                                                                 
 max_pooling1d_9 (MaxPoolin  (None, 255, 32)           0         
 g1D)                                                            
                                                                 
 conv1d_10 (Conv1D)          (None, 253, 64)           6208      
                                                                 
 max_pooling1d_10 (MaxPooli  (None, 126, 64)           0         
 ng1D)                                                           
                                                                 
 flatten_6 (Flatten)         (None, 806

# **L1 AND L2**

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Conv1D, Dense
from tensorflow.keras.regularizers import l1_l2

# Define the path where your model is saved
model_path = '/content/drive/MyDrive/Depression/Model/finetune_on_latest.h5'

# Load the model
fine_tuned_model = load_model(model_path)

# Define regularization strength
l1_value = 0.1
l2_value = 0.1

# Apply L1 and L2 regularization to each layer
for layer in fine_tuned_model.layers:
    if isinstance(layer, Conv1D) or isinstance(layer, Dense):
        layer.kernel_regularizer = l1_l2(l1=l1_value, l2=l2_value)

# Compile the model
fine_tuned_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define the path where your updated model will be saved
save_path = "/content/drive/MyDrive/Depression/Model/l1.h5"

# Save the updated model
fine_tuned_model.save(save_path)

print("Updated model with L1 and L2 regularization applied has been saved as 'l1.h5'")


Updated model with L1 and L2 regularization applied has been saved as 'l1.h5'


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.models import load_model

# Load the saved model
model_save_path = '/content/drive/MyDrive/Depression/Model/l1.h5'  # Update with your model's path
loaded_model = load_model(model_save_path)
print("Model loaded successfully.")

# Load and prepare the test data
test_data_path = '/content/drive/MyDrive/Depression/Android Dataset/Interview_test.csv'  # Update with your test data's path
test_data = pd.read_csv(test_data_path)

# Assuming your CSV has specific columns to drop (like 'name') and a 'Class' column for labels
X_test = test_data.drop(columns=['Class', 'name']).values
y_test = test_data['Class'].values

# Scale the features
scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(X_test)

# Reshape the data if your model expects a specific input shape (example for CNN)
X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

# Make predictions on the test data
y_pred = (loaded_model.predict(X_test_scaled) > 0.5).astype(int)

# Calculate and print the classification report and accuracy
print("CNN Model Classification Report for Testing Set:")
print(classification_report(y_test, y_pred))
accuracy_test = accuracy_score(y_test, y_pred)
print(f"\nAccuracy for CNN Testing Set: {accuracy_test:.4f}")


Model loaded successfully.
47/47 [==============================] - 3s 5ms/step
CNN Model Classification Report for Testing Set:
              precision    recall  f1-score   support

           0       0.55      0.91      0.68       624
           1       0.88      0.46      0.61       875

    accuracy                           0.65      1499
   macro avg       0.71      0.69      0.65      1499
weighted avg       0.74      0.65      0.64      1499


Accuracy for CNN Testing Set: 0.6504


# **EWC**

In [ ]:
import tensorflow as tf
import pandas as pd

# Define the function to compute the EWC penalty
def ewc_penalty(model, precision_matrices):
    ewc_loss = 0
    initial_weights = model.get_weights()
    for j, var in enumerate(model.trainable_variables):
        ewc_loss += tf.reduce_sum(precision_matrices[j] * tf.square(var - initial_weights[j]))
    return ewc_loss


def train_with_ewc(model, task_B_train, precision_matrices, epochs=6):
    for epoch in range(epochs):
        for batch_data in task_B_train:
            imgs = batch_data.iloc[:, 1:-1].to_numpy()  # Exclude 'name' and 'Class' columns
            labels = batch_data['Class'].to_numpy()
            with tf.GradientTape() as tape:
                preds = model(imgs)
                loss = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(labels, preds))
                ewc_loss = ewc_penalty(model, precision_matrices)
                total_loss = loss + ewc_loss
            grads = tape.gradient(total_loss, model.trainable_variables)
            model.optimizer.apply_gradients(zip(grads, model.trainable_variables))


def compute_precision_matrices(model, task_set, num_batches=1, batch_size=32):
    precision_matrices = {n: tf.zeros_like(p.value()) for n, p in enumerate(model.trainable_variables)}

    for _ in range(num_batches):
        task_set_shuffled = task_set.sample(frac=1)  # Shuffle the dataset
        for start in range(0, len(task_set_shuffled), batch_size):
            batch_data = task_set_shuffled[start:start+batch_size]
            imgs = batch_data.iloc[:, 1:-1].to_numpy()  # Exclude 'name' and 'Class' columns
            labels = batch_data['Class'].to_numpy()
            with tf.GradientTape() as tape:
                preds = model(imgs)
                ll = tf.nn.log_softmax(preds)
            ll_grads = tape.gradient(ll, model.trainable_variables)
            for j, g in enumerate(ll_grads):
                precision_matrices[j] += tf.reduce_mean(tf.square(g), axis=0) / num_batches

    return precision_matrices


# Load the pre-trained model
pretrained_model_path = '/content/drive/MyDrive/Depression/Model/l1.h5'
pretrained_model = tf.keras.models.load_model(pretrained_model_path)

# Load dataset A
task_A_train_path = '/content/drive/MyDrive/Depression/EDAIC Dataset/training_labels.csv'
task_A_train_data = pd.read_csv(task_A_train_path)

# Load dataset B
task_B_train_path = '/content/drive/MyDrive/Depression/Android Dataset/Interview_train.csv'
task_B_train_data = pd.read_csv(task_B_train_path)

# Compute Fisher information matrix and precision matrices for dataset A
precision_matrices_A = compute_precision_matrices(pretrained_model, task_A_train_data, num_batches=100)

# Shuffle dataset B
task_B_train_data_shuffled = task_B_train_data.sample(frac=1)

# Compute Fisher information matrix and precision matrices for dataset B
precision_matrices_B = compute_precision_matrices(pretrained_model, task_B_train_data_shuffled, num_batches=100)

# Fine-tune the model on dataset B with EWC
train_with_ewc(pretrained_model, [task_B_train_data_shuffled], precision_matrices_B, epochs=5)

# Save the model
pretrained_model.save('/content/drive/MyDrive/Depression/Model/ewc_model.h5')
print("Model trained with EWC has been saved as 'ewc_model.h5'")


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model trained with EWC has been saved as 'ewc_model.h5'


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.models import load_model

# Load the saved model
model_save_path = '/content/drive/MyDrive/Depression/Model/ewc_model.h5'  # Update with your model's path
loaded_model = load_model(model_save_path)
print("Model loaded successfully.")

# Load and prepare the test data
test_data_path = '/content/drive/MyDrive/Depression/Android Dataset/Interview_test.csv'  # Update with your test data's path
test_data = pd.read_csv(test_data_path)

# Assuming your CSV has specific columns to drop (like 'name') and a 'Class' column for labels
X_test = test_data.drop(columns=['Class', 'name']).values
y_test = test_data['Class'].values

# Scale the features
scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(X_test)

# Reshape the data if your model expects a specific input shape (example for CNN)
X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

# Make predictions on the test data
y_pred = (loaded_model.predict(X_test_scaled) > 0.5).astype(int)

# Calculate and print the classification report and accuracy
print("CNN Model Classification Report for Testing Set:")
print(classification_report(y_test, y_pred))
accuracy_test = accuracy_score(y_test, y_pred)
print(f"\nAccuracy for CNN Testing Set: {accuracy_test:.4f}")


Model loaded successfully.
47/47 [==============================] - 0s 2ms/step
CNN Model Classification Report for Testing Set:
              precision    recall  f1-score   support

           0       0.42      1.00      0.59       624
           1       0.00      0.00      0.00       875

    accuracy                           0.42      1499
   macro avg       0.21      0.50      0.29      1499
weighted avg       0.17      0.42      0.24      1499


Accuracy for CNN Testing Set: 0.4163


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import pandas as pd
import inspect  # Add this import

# Load the dataset
task_A_train_path = '/content/drive/MyDrive/Depression/Android Dataset/Reading_train.csv'
task_A_train_data = pd.read_csv(task_A_train_path)

# Print information about the dataset
print("Information about the dataset:")
print(task_A_train_data.info())

# Print the first few rows of the dataset
print("\nFirst few rows of the dataset:")
print(task_A_train_data.head())

# Print the compute_precision_matrices function definition
def compute_precision_matrices(model, task_set, num_batches=1, batch_size=32):
    print("\ncompute_precision_matrices function definition:")
    print(inspect.getsource(compute_precision_matrices))

# Call the compute_precision_matrices function
compute_precision_matrices(pretrained_model, task_A_train_data, num_batches=100)


Information about the dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 790 entries, 0 to 789
Columns: 514 entries, name to feature_511
dtypes: float64(512), int64(1), object(1)
memory usage: 3.1+ MB
None

First few rows of the dataset:
                        name  Class  feature_0  feature_1  feature_2  \
0  Reading_depressed_151.wav      1 -18.882141  -8.257141  10.337055   
1  Reading_depressed_152.wav      1 -22.440567  -7.324803  12.972845   
2  Reading_depressed_153.wav      1 -24.144209   5.473886   6.452698   
3  Reading_depressed_154.wav      1 -23.029322   8.752594   7.679756   
4  Reading_depressed_155.wav      1 -20.563568  10.359172  13.883033   

   feature_3  feature_4  feature_5  feature_6  feature_7  ...  feature_502  \
0  14.460301   0.022575 -12.631349 -19.638437  -1.236870  ...   -21.927385   
1  12.255430   0.514035 -12.671404 -18.478327  -6.821404  ...   -25.670532   
2   4.345125  -0.957478  -4.620906 -21.235149  14.255901  ...   -25.407789   
3  14.045

# **extraas**

In [ ]:
import tensorflow as tf

# Load the model from the .h5 file
model_path = "/content/drive/MyDrive/latest_CNN.h5"
loaded_model = tf.keras.models.load_model(model_path)

# Display the architecture and parameters of the loaded model
loaded_model.summary()


Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 512, 1)]          0         
                                                                 
 conv1d_9 (Conv1D)           (None, 510, 32)           128       
                                                                 
 max_pooling1d_9 (MaxPoolin  (None, 255, 32)           0         
 g1D)                                                            
                                                                 
 conv1d_10 (Conv1D)          (None, 253, 64)           6208      
                                                                 
 max_pooling1d_10 (MaxPooli  (None, 126, 64)           0         
 ng1D)                                                           
                                                                 
 flatten_6 (Flatten)         (None, 8064)              0   

In [ ]:
import pandas as pd

# Load your dataset
dataset_path = "/content/drive/MyDrive/Depression/EDAIC Dataset/training_labels.csv"  # Update with your dataset path
dataset = pd.read_csv(dataset_path)

# Assuming the features are all columns except the label column
features = dataset.drop(columns=['Class'])

# Get the shape of the features
input_shape = features.shape[1:]  # Exclude the first dimension which represents the number of samples

print("Input shape:", input_shape)


Input shape: (513,)


In [ ]:
import pandas as pd

# Define the path to your CSV file
csv_file_path = "/content/drive/MyDrive/Depression/EDAIC Dataset/training_labels.csv"

# Read the CSV file into a pandas DataFrame
dataframe = pd.read_csv(csv_file_path)

# Get the shape of the DataFrame
shape = dataframe.shape

# Print the shape of the DataFrame
print("Shape of the DataFrame:", shape)


Shape of the DataFrame: (12547, 514)


# **EWC-2 Testing**

In [ ]:
import tensorflow as tf
import pandas as pd

# Define the function to compute the EWC penalty
def ewc_penalty(model, precision_matrices):
    ewc_loss = 0
    initial_weights = model.get_weights()
    for j, var in enumerate(model.trainable_variables):
        ewc_loss += tf.reduce_sum(precision_matrices[j] * tf.square(var - initial_weights[j]))
    return ewc_loss

def train_with_ewc(model, task_B_train, precision_matrices, epochs=6):
    for epoch in range(epochs):
        for batch_data in task_B_train:
            imgs = batch_data.iloc[:, 1:-1].to_numpy()  # Exclude 'name' and 'Class' columns
            labels = batch_data['Class'].to_numpy()
            with tf.GradientTape() as tape:
                preds = model(imgs)
                loss = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(labels, preds))
                ewc_loss = ewc_penalty(model, precision_matrices)
                total_loss = loss + ewc_loss
            grads = tape.gradient(total_loss, model.trainable_variables)
            model.optimizer.apply_gradients(zip(grads, model.trainable_variables))


def compute_precision_matrices(model, task_set, num_batches=1, batch_size=32):
    precision_matrices = {n: tf.zeros_like(p.value()) for n, p in enumerate(model.trainable_variables)}

    for _ in range(num_batches):
        task_set_shuffled = task_set.sample(frac=1)  # Shuffle the dataset
        for start in range(0, len(task_set_shuffled), batch_size):
            batch_data = task_set_shuffled[start:start+batch_size]
            imgs = batch_data.iloc[:, 1:-1].to_numpy()  # Exclude 'name' and 'Class' columns
            labels = batch_data['Class'].to_numpy()
            with tf.GradientTape() as tape:
                preds = model(imgs)
                ll = tf.nn.log_softmax(preds)
            ll_grads = tape.gradient(ll, model.trainable_variables)
            for j, g in enumerate(ll_grads):
                precision_matrices[j] += tf.reduce_mean(tf.square(g), axis=0) / num_batches

    return precision_matrices


# Load the pre-trained model
pretrained_model_path = '/content/drive/MyDrive/Depression/Model/l1.h5'
pretrained_model = tf.keras.models.load_model(pretrained_model_path)

# Load dataset A (Interview)
train_csv_path_A = '/content/drive/MyDrive/Depression/Android Dataset/Interview_train.csv'
test_csv_path_A = '/content/drive/MyDrive/Depression/Android Dataset/Interview_test.csv'
task_A_train_data = pd.read_csv(train_csv_path_A)
task_A_test_data = pd.read_csv(test_csv_path_A)

# Load dataset B (Reading)
train_csv_path_B = '/content/drive/MyDrive/Depression/Android Dataset/Reading_train.csv'
test_csv_path_B = '/content/drive/MyDrive/Depression/Android Dataset/Reading_test.csv'
task_B_train_data = pd.read_csv(train_csv_path_B)
task_B_test_data = pd.read_csv(test_csv_path_B)

# Compute Fisher information matrix and precision matrices for dataset A
precision_matrices_A = compute_precision_matrices(pretrained_model, task_A_train_data, num_batches=100)

# Compute Fisher information matrix and precision matrices for dataset B
precision_matrices_B = compute_precision_matrices(pretrained_model, task_B_train_data, num_batches=100)

# Fine-tune the model on dataset B with EWC
train_with_ewc(pretrained_model, [task_B_train_data], precision_matrices_B, epochs=5)

# Save the model
pretrained_model.save('/content/drive/MyDrive/Depression/Model/ewc_model_2.h5')
print("Model trained with EWC has been saved as 'ewc_model.h5'")


Model trained with EWC has been saved as 'ewc_model.h5'


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.models import load_model

# Load the saved model
model_save_path = '/content/drive/MyDrive/Depression/Model/ewc_model_2.h5'  # Update with your model's path
loaded_model = load_model(model_save_path)
print("Model loaded successfully.")

# Load and prepare the test data
test_data_path = '/content/drive/MyDrive/Depression/Android Dataset/Reading_test.csv'  # Update with your test data's path
test_data = pd.read_csv(test_data_path)

# Assuming your CSV has specific columns to drop (like 'name') and a 'Class' column for labels
X_test = test_data.drop(columns=['Class', 'name']).values
y_test = test_data['Class'].values

# Scale the features
scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(X_test)

# Reshape the data if your model expects a specific input shape (example for CNN)
X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

# Make predictions on the test data
y_pred = (loaded_model.predict(X_test_scaled) > 0.5).astype(int)

# Calculate and print the classification report and accuracy
print("CNN Model Classification Report for Testing Set:")
print(classification_report(y_test, y_pred))
accuracy_test = accuracy_score(y_test, y_pred)
print(f"\nAccuracy for CNN Testing Set: {accuracy_test:.4f}")


Model loaded successfully.
8/8 [==============================] - 0s 4ms/step
CNN Model Classification Report for Testing Set:
              precision    recall  f1-score   support

           0       0.40      1.00      0.58       102
           1       0.00      0.00      0.00       150

    accuracy                           0.40       252
   macro avg       0.20      0.50      0.29       252
weighted avg       0.16      0.40      0.23       252


Accuracy for CNN Testing Set: 0.4048


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# **corrections**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Function to load and preprocess test data
def load_and_preprocess_test_data(test_data_path):
    test_data = pd.read_csv(test_data_path)
    X_test = test_data.drop(columns=['Class', 'name']).values
    y_test = test_data['Class'].values
    scaler = StandardScaler()
    X_test_scaled = scaler.fit_transform(X_test)
    X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))
    return X_test_scaled, y_test

# Function to evaluate model on test data
def evaluate_model(model, X_test, y_test):
    y_pred = (model.predict(X_test) > 0.5).astype(int)
    print("Model Classification Report for Testing Set:")
    print(classification_report(y_test, y_pred))
    accuracy_test = accuracy_score(y_test, y_pred)
    print(f"\nAccuracy for Testing Set: {accuracy_test:.4f}")

# Load the saved model
model_save_path = '/content/drive/MyDrive/Depression/Model/latest_CNN.h5'
loaded_model = load_model(model_save_path)
print("Model loaded successfully.")

# Evaluate on first test dataset
test_data_path_1 = '/content/drive/MyDrive/Depression/EDAIC Dataset/testing_labels.csv'
X_test_1, y_test_1 = load_and_preprocess_test_data(test_data_path_1)
evaluate_model(loaded_model, X_test_1, y_test_1)

# Evaluate on second test dataset
test_data_path_2 = '/content/drive/MyDrive/Depression/Android Dataset/Interview_test.csv'
X_test_2, y_test_2 = load_and_preprocess_test_data(test_data_path_2)
evaluate_model(loaded_model, X_test_2, y_test_2)

# Fine-tuning the model on Interview dataset
interview_train_path = '/content/drive/MyDrive/Depression/Android Dataset/Interview_train.csv'
interview_train_data = pd.read_csv(interview_train_path)
X_interview = interview_train_data.drop(columns=['name', 'Class']).values
y_interview = interview_train_data['Class'].values

scaler = StandardScaler()
X_interview_scaled = scaler.fit_transform(X_interview)

# Reshape data for CNN
X_interview_scaled_reshaped = X_interview_scaled.reshape((X_interview_scaled.shape[0], X_interview_scaled.shape[1], 1))

# Compile model with smaller learning rate
loaded_model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Use early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model on Interview dataset
loaded_model.fit(X_interview_scaled_reshaped, y_interview, epochs=10, validation_split=0.2, callbacks=[early_stopping])

# Save the fine-tuned model
loaded_model.save('/content/drive/MyDrive/Depression/Model/finetuned_model.h5')

# Evaluate the fine-tuned model on test datasets
evaluate_model(loaded_model, X_test_1, y_test_1)
evaluate_model(loaded_model, X_test_2, y_test_2)


Model loaded successfully.
77/77 [==============================] - 4s 6ms/step
Model Classification Report for Testing Set:
              precision    recall  f1-score   support

           0       0.78      0.95      0.86      1897
           1       0.26      0.06      0.10       549

    accuracy                           0.75      2446
   macro avg       0.52      0.51      0.48      2446
weighted avg       0.66      0.75      0.69      2446


Accuracy for Testing Set: 0.7522
47/47 [==============================] - 0s 6ms/step
Model Classification Report for Testing Set:
              precision    recall  f1-score   support

           0       0.42      0.94      0.58       624
           1       0.62      0.07      0.13       875

    accuracy                           0.43      1499
   macro avg       0.52      0.51      0.35      1499
weighted avg       0.54      0.43      0.31      1499


Accuracy for Testing Set: 0.4323
Epoch 1/10
85/85 [==============================] - 10s

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


77/77 [==============================] - 0s 2ms/step
Model Classification Report for Testing Set:
              precision    recall  f1-score   support

           0       0.81      0.65      0.72      1897
           1       0.28      0.47      0.35       549

    accuracy                           0.61      2446
   macro avg       0.54      0.56      0.54      2446
weighted avg       0.69      0.61      0.64      2446


Accuracy for Testing Set: 0.6112
47/47 [==============================] - 0s 2ms/step
Model Classification Report for Testing Set:
              precision    recall  f1-score   support

           0       0.58      0.84      0.69       624
           1       0.83      0.57      0.68       875

    accuracy                           0.68      1499
   macro avg       0.71      0.70      0.68      1499
weighted avg       0.73      0.68      0.68      1499


Accuracy for Testing Set: 0.6805


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

# Function to load and preprocess test data
def load_and_preprocess_test_data(test_data_path):
    test_data = pd.read_csv(test_data_path)
    X_test = test_data.drop(columns=['Class', 'name']).values
    y_test = test_data['Class'].values
    scaler = StandardScaler()
    X_test_scaled = scaler.fit_transform(X_test)
    X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))
    return X_test_scaled, y_test

# Function to evaluate model on test data
def evaluate_model(model, X_test, y_test):
    y_pred = (model.predict(X_test) > 0.5).astype(int)
    print("Model Classification Report for Testing Set:")
    print(classification_report(y_test, y_pred))
    accuracy_test = accuracy_score(y_test, y_pred)
    print(f"\nAccuracy for Testing Set: {accuracy_test:.4f}")

# Load the saved model
model_save_path = '/content/drive/MyDrive/Depression/Model/latest_CNN.h5'
loaded_model = load_model(model_save_path)
print("Model loaded successfully.")

# Evaluate on first test dataset
test_data_path_1 = '/content/drive/MyDrive/Depression/EDAIC Dataset/testing_labels.csv'
X_test_1, y_test_1 = load_and_preprocess_test_data(test_data_path_1)
evaluate_model(loaded_model, X_test_1, y_test_1)

# Evaluate on second test dataset
test_data_path_2 = '/content/drive/MyDrive/Depression/Android Dataset/Interview_test.csv'
X_test_2, y_test_2 = load_and_preprocess_test_data(test_data_path_2)
evaluate_model(loaded_model, X_test_2, y_test_2)

# Fine-tuning the model on Interview dataset
interview_train_path = '/content/drive/MyDrive/Depression/Android Dataset/Interview_train.csv'
interview_train_data = pd.read_csv(interview_train_path)
X_interview = interview_train_data.drop(columns=['name', 'Class']).values
y_interview = interview_train_data['Class'].values

scaler = StandardScaler()
X_interview_scaled = scaler.fit_transform(X_interview)

# Reshape data for CNN
X_interview_scaled_reshaped = X_interview_scaled.reshape((X_interview_scaled.shape[0], X_interview_scaled.shape[1], 1))

# Compile model with smaller learning rate
loaded_model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Use early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model on Interview dataset
loaded_model.fit(X_interview_scaled_reshaped, y_interview, epochs=10, validation_split=0.2, callbacks=[early_stopping])

# Save the fine-tuned model
loaded_model.save('/content/drive/MyDrive/Depression/Model/finetuned_model.h5')

# Evaluate the fine-tuned model on test datasets
evaluate_model(loaded_model, X_test_1, y_test_1)
evaluate_model(loaded_model, X_test_2, y_test_2)

# Applying L1 and L2 Regularization
l1_value = 0.1
l2_value = 0.1

for layer in loaded_model.layers:
    if isinstance(layer, tf.keras.layers.Conv1D) or isinstance(layer, tf.keras.layers.Dense):
        layer.kernel_regularizer = tf.keras.regularizers.l1_l2(l1=l1_value, l2=l2_value)

# Save the model with regularization applied
loaded_model.save('/content/drive/MyDrive/Depression/Model/l1_regularized_model.h5')
print("Model with L1 and L2 regularization applied has been saved.")

# Load the test data for L1 regularized model
test_data_path_l1 = '/content/drive/MyDrive/Depression/Android Dataset/Interview_test.csv'
X_test_l1, y_test_l1 = load_and_preprocess_test_data(test_data_path_l1)

# Evaluate the L1 regularized model on test data
evaluate_model(loaded_model, X_test_l1, y_test_l1)

# Elastic Weight Consolidation (EWC)
def ewc_penalty(model, precision_matrices):
    ewc_loss = 0
    initial_weights = model.get_weights()
    for j, var in enumerate(model.trainable_variables):
        ewc_loss += tf.reduce_sum(precision_matrices[j] * tf.square(var - initial_weights[j]))
    return ewc_loss

def train_with_ewc(model, task_B_train, precision_matrices, epochs=6):
    for epoch in range(epochs):
        for batch_data in task_B_train:
            imgs = batch_data.iloc[:, 1:-1].to_numpy()  # Exclude 'name' and 'Class' columns
            labels = batch_data['Class'].to_numpy()
            with tf.GradientTape() as tape:
                preds = model(imgs)
                loss = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(labels, preds))
                ewc_loss = ewc_penalty(model, precision_matrices)
                total_loss = loss + ewc_loss
            grads = tape.gradient(total_loss, model.trainable_variables)
            model.optimizer.apply_gradients(zip(grads, model.trainable_variables))

def compute_precision_matrices(model, task_set, num_batches=1, batch_size=32):
    precision_matrices = {n: tf.zeros_like(p.value()) for n, p in enumerate(model.trainable_variables)}

    for _ in range(num_batches):
        task_set_shuffled = task_set.sample(frac=1)  # Shuffle the dataset
        for start in range(0, len(task_set_shuffled), batch_size):
            batch_data = task_set_shuffled[start:start+batch_size]
            imgs = batch_data.iloc[:, 1:-1].to_numpy()  # Exclude 'name' and 'Class' columns
            labels = batch_data['Class'].to_numpy()
            with tf.GradientTape() as tape:
                preds = model(imgs)
                ll = tf.nn.log_softmax(preds)
            ll_grads = tape.gradient(ll, model.trainable_variables)
            for j, g in enumerate(ll_grads):
                precision_matrices[j] += tf.reduce_mean(tf.square(g), axis=0) / num_batches

    return precision_matrices

# Load the pre-trained model
pretrained_model_path = '/content/drive/MyDrive/Depression/Model/l1_regularized_model.h5'
pretrained_model = tf.keras.models.load_model(pretrained_model_path)

# Load dataset A
task_A_train_path = '/content/drive/MyDrive/Depression/EDAIC Dataset/training_labels.csv'
task_A_train_data = pd.read_csv(task_A_train_path)

# Load dataset B
task_B_train_path = '/content/drive/MyDrive/Depression/Android Dataset/Interview_train.csv'
task_B_train_data = pd.read_csv(task_B_train_path)

# Compute Fisher information matrix and precision matrices for dataset A
precision_matrices_A = compute_precision_matrices(pretrained_model, task_A_train_data, num_batches=100)

# Shuffle dataset B
task_B_train_data_shuffled = task_B_train_data.sample(frac=1)

# Compute Fisher information matrix and precision matrices for dataset B
precision_matrices_B = compute_precision_matrices(pretrained_model, task_B_train_data_shuffled, num_batches=100)

# Fine-tune the model on dataset B with EWC
train_with_ewc(pretrained_model, [task_B_train_data_shuffled], precision_matrices_B, epochs=5)

# Save the model
pretrained_model.save('/content/drive/MyDrive/Depression/Model/ewc_model.h5')
print("Model trained with EWC has been saved as 'ewc_model.h5'")

# Load the saved EWC model
ewc_model_save_path = '/content/drive/MyDrive/Depression/Model/ewc_model.h5'
ewc_loaded_model = load_model(ewc_model_save_path)
print("EWC Model loaded successfully.")

# Evaluate the EWC model on test data
evaluate_model(ewc_loaded_model, X_test_l1, y_test_l1)


Model loaded successfully.
77/77 [==============================] - 0s 3ms/step
Model Classification Report for Testing Set:
              precision    recall  f1-score   support

           0       0.78      0.95      0.86      1897
           1       0.26      0.06      0.10       549

    accuracy                           0.75      2446
   macro avg       0.52      0.51      0.48      2446
weighted avg       0.66      0.75      0.69      2446


Accuracy for Testing Set: 0.7522
47/47 [==============================] - 0s 3ms/step
Model Classification Report for Testing Set:
              precision    recall  f1-score   support

           0       0.42      0.94      0.58       624
           1       0.62      0.07      0.13       875

    accuracy                           0.43      1499
   macro avg       0.52      0.51      0.35      1499
weighted avg       0.54      0.43      0.31      1499


Accuracy for Testing Set: 0.4323
Epoch 1/10
85/85 [==============================] - 4s 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


77/77 [==============================] - 0s 2ms/step
Model Classification Report for Testing Set:
              precision    recall  f1-score   support

           0       0.80      0.56      0.66      1897
           1       0.26      0.52      0.34       549

    accuracy                           0.55      2446
   macro avg       0.53      0.54      0.50      2446
weighted avg       0.68      0.55      0.59      2446


Accuracy for Testing Set: 0.5507
47/47 [==============================] - 0s 2ms/step
Model Classification Report for Testing Set:
              precision    recall  f1-score   support

           0       0.56      0.71      0.62       624
           1       0.74      0.60      0.66       875

    accuracy                           0.64      1499
   macro avg       0.65      0.65      0.64      1499
weighted avg       0.66      0.64      0.65      1499


Accuracy for Testing Set: 0.6431
Model with L1 and L2 regularization applied has been saved.


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


47/47 [==============================] - 0s 2ms/step
Model Classification Report for Testing Set:
              precision    recall  f1-score   support

           0       0.56      0.71      0.62       624
           1       0.74      0.60      0.66       875

    accuracy                           0.64      1499
   macro avg       0.65      0.65      0.64      1499
weighted avg       0.66      0.64      0.65      1499


Accuracy for Testing Set: 0.6431


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model trained with EWC has been saved as 'ewc_model.h5'
EWC Model loaded successfully.
47/47 [==============================] - 0s 2ms/step
Model Classification Report for Testing Set:
              precision    recall  f1-score   support

           0       0.42      1.00      0.59       624
           1       0.00      0.00      0.00       875

    accuracy                           0.42      1499
   macro avg       0.21      0.50      0.29      1499
weighted avg       0.17      0.42      0.24      1499


Accuracy for Testing Set: 0.4163


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

# Function to load and preprocess test data
def load_and_preprocess_test_data(test_data_path):
    test_data = pd.read_csv(test_data_path)
    X_test = test_data.drop(columns=['Class', 'name']).values
    y_test = test_data['Class'].values
    scaler = StandardScaler()
    X_test_scaled = scaler.fit_transform(X_test)
    X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))
    return X_test_scaled, y_test

# Function to evaluate model on test data
def evaluate_model(model, X_test, y_test):
    y_pred = (model.predict(X_test) > 0.5).astype(int)
    print("Model Classification Report for Testing Set:")
    print(classification_report(y_test, y_pred))
    accuracy_test = accuracy_score(y_test, y_pred)
    print(f"\nAccuracy for Testing Set: {accuracy_test:.4f}")

# Load the saved model
model_save_path = '/content/drive/MyDrive/Depression/Model/latest_CNN.h5'
loaded_model = load_model(model_save_path)
print("Model loaded successfully.")

# Evaluate on first test dataset
test_data_path_1 = '/content/drive/MyDrive/Depression/EDAIC Dataset/testing_labels.csv'
X_test_1, y_test_1 = load_and_preprocess_test_data(test_data_path_1)
evaluate_model(loaded_model, X_test_1, y_test_1)

# Evaluate on second test dataset
test_data_path_2 = '/content/drive/MyDrive/Depression/Android Dataset/Interview_test.csv'
X_test_2, y_test_2 = load_and_preprocess_test_data(test_data_path_2)
evaluate_model(loaded_model, X_test_2, y_test_2)

# Fine-tuning the model on Interview dataset
interview_train_path = '/content/drive/MyDrive/Depression/Android Dataset/Interview_train.csv'
interview_train_data = pd.read_csv(interview_train_path)
X_interview = interview_train_data.drop(columns=['name', 'Class']).values
y_interview = interview_train_data['Class'].values

scaler = StandardScaler()
X_interview_scaled = scaler.fit_transform(X_interview)

# Reshape data for CNN
X_interview_scaled_reshaped = X_interview_scaled.reshape((X_interview_scaled.shape[0], X_interview_scaled.shape[1], 1))

# Compile model with smaller learning rate
loaded_model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Use early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model on Interview dataset
loaded_model.fit(X_interview_scaled_reshaped, y_interview, epochs=10, validation_split=0.2, callbacks=[early_stopping])

# Save the fine-tuned model
loaded_model.save('/content/drive/MyDrive/Depression/Model/finetuned_model.h5')

# Evaluate the fine-tuned model on test datasets
evaluate_model(loaded_model, X_test_1, y_test_1)
evaluate_model(loaded_model, X_test_2, y_test_2)

# Applying L1 and L2 Regularization
l1_value = 0.1
l2_value = 0.1

for layer in loaded_model.layers:
    if isinstance(layer, tf.keras.layers.Conv1D) or isinstance(layer, tf.keras.layers.Dense):
        layer.kernel_regularizer = tf.keras.regularizers.l1_l2(l1=l1_value, l2=l2_value)

# Save the model with regularization applied
loaded_model.save('/content/drive/MyDrive/Depression/Model/l1_regularized_model.h5')
print("Model with L1 and L2 regularization applied has been saved.")

# Load the test data for L1 regularized model
test_data_path_l1 = '/content/drive/MyDrive/Depression/Android Dataset/Interview_test.csv'
X_test_l1, y_test_l1 = load_and_preprocess_test_data(test_data_path_l1)

# Evaluate the L1 regularized model on test data
evaluate_model(loaded_model, X_test_l1, y_test_l1)

# Elastic Weight Consolidation (EWC)
def ewc_penalty(model, precision_matrices):
    ewc_loss = 0
    initial_weights = model.get_weights()
    for j, var in enumerate(model.trainable_variables):
        ewc_loss += tf.reduce_sum(precision_matrices[j] * tf.square(var - initial_weights[j]))
    return ewc_loss

def train_with_ewc(model, task_B_train, precision_matrices, epochs=6):
    for epoch in range(epochs):
        for batch_data in task_B_train:
            imgs = batch_data.iloc[:, 1:-1].to_numpy()  # Exclude 'name' and 'Class' columns
            labels = batch_data['Class'].to_numpy()
            with tf.GradientTape() as tape:
                preds = model(imgs)
                loss = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(labels, preds))
                ewc_loss = ewc_penalty(model, precision_matrices)
                total_loss = loss + ewc_loss
            grads = tape.gradient(total_loss, model.trainable_variables)
            model.optimizer.apply_gradients(zip(grads, model.trainable_variables))

def compute_precision_matrices(model, task_set, num_batches=1, batch_size=32):
    precision_matrices = {n: tf.zeros_like(p.value()) for n, p in enumerate(model.trainable_variables)}

    for _ in range(num_batches):
        task_set_shuffled = task_set.sample(frac=1)  # Shuffle the dataset
        for start in range(0, len(task_set_shuffled), batch_size):
            batch_data = task_set_shuffled[start:start+batch_size]
            imgs = batch_data.iloc[:, 1:-1].to_numpy()  # Exclude 'name' and 'Class' columns
            labels = batch_data['Class'].to_numpy()
            with tf.GradientTape() as tape:
                preds = model(imgs)
                ll = tf.nn.log_softmax(preds)
            ll_grads = tape.gradient(ll, model.trainable_variables)
            for j, g in enumerate(ll_grads):
                precision_matrices[j] += tf.reduce_mean(tf.square(g), axis=0) / num_batches

    return precision_matrices

# Load the pre-trained model
pretrained_model_path = '/content/drive/MyDrive/Depression/Model/l1_regularized_model.h5'
pretrained_model = tf.keras.models.load_model(pretrained_model_path)

# Load dataset A
task_A_train_path = '/content/drive/MyDrive/Depression/EDAIC Dataset/training_labels.csv'
task_A_train_data = pd.read_csv(task_A_train_path)

# Load dataset B
task_B_train_path = '/content/drive/MyDrive/Depression/Android Dataset/Interview_train.csv'
task_B_train_data = pd.read_csv(task_B_train_path)

# Compute Fisher information matrix and precision matrices for dataset A
precision_matrices_A = compute_precision_matrices(pretrained_model, task_A_train_data, num_batches=100)

# Shuffle dataset B
task_B_train_data_shuffled = task_B_train_data.sample(frac=1)

# Compute Fisher information matrix and precision matrices for dataset B
precision_matrices_B = compute_precision_matrices(pretrained_model, task_B_train_data_shuffled, num_batches=100)

# Fine-tune the model on dataset B with EWC
train_with_ewc(pretrained_model, [task_B_train_data_shuffled], precision_matrices_B, epochs=5)

# Save the model
pretrained_model.save('/content/drive/MyDrive/Depression/Model/ewc_model.h5')
print("Model trained with EWC has been saved as 'ewc_model.h5'")

# Load the saved EWC model
ewc_model_save_path = '/content/drive/MyDrive/Depression/Model/ewc_model.h5'
ewc_loaded_model = load_model(ewc_model_save_path)
print("EWC Model loaded successfully.")

# Evaluate the EWC model on test data
evaluate_model(ewc_loaded_model, X_test_l1, y_test_l1)


Model loaded successfully.
77/77 [==============================] - 4s 9ms/step
Model Classification Report for Testing Set:
              precision    recall  f1-score   support

           0       0.78      0.95      0.86      1897
           1       0.26      0.06      0.10       549

    accuracy                           0.75      2446
   macro avg       0.52      0.51      0.48      2446
weighted avg       0.66      0.75      0.69      2446


Accuracy for Testing Set: 0.7522
47/47 [==============================] - 0s 7ms/step
Model Classification Report for Testing Set:
              precision    recall  f1-score   support

           0       0.42      0.94      0.58       624
           1       0.62      0.07      0.13       875

    accuracy                           0.43      1499
   macro avg       0.52      0.51      0.35      1499
weighted avg       0.54      0.43      0.31      1499


Accuracy for Testing Set: 0.4323
Epoch 1/10
85/85 [==============================] - 11s

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


77/77 [==============================] - 0s 2ms/step
Model Classification Report for Testing Set:
              precision    recall  f1-score   support

           0       0.81      0.63      0.71      1897
           1       0.28      0.50      0.36       549

    accuracy                           0.60      2446
   macro avg       0.55      0.56      0.53      2446
weighted avg       0.69      0.60      0.63      2446


Accuracy for Testing Set: 0.5989
47/47 [==============================] - 0s 2ms/step
Model Classification Report for Testing Set:
              precision    recall  f1-score   support

           0       0.59      0.82      0.68       624
           1       0.82      0.59      0.68       875

    accuracy                           0.68      1499
   macro avg       0.70      0.70      0.68      1499
weighted avg       0.72      0.68      0.68      1499


Accuracy for Testing Set: 0.6838


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model with L1 and L2 regularization applied has been saved.
47/47 [==============================] - 0s 2ms/step
Model Classification Report for Testing Set:
              precision    recall  f1-score   support

           0       0.59      0.82      0.68       624
           1       0.82      0.59      0.68       875

    accuracy                           0.68      1499
   macro avg       0.70      0.70      0.68      1499
weighted avg       0.72      0.68      0.68      1499


Accuracy for Testing Set: 0.6838


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model trained with EWC has been saved as 'ewc_model.h5'
EWC Model loaded successfully.
47/47 [==============================] - 0s 2ms/step
Model Classification Report for Testing Set:
              precision    recall  f1-score   support

           0       0.42      1.00      0.59       624
           1       0.00      0.00      0.00       875

    accuracy                           0.42      1499
   macro avg       0.21      0.50      0.29      1499
weighted avg       0.17      0.42      0.24      1499


Accuracy for Testing Set: 0.4163


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the evaluate function
def evaluate(model, X, y):
    y_pred = np.argmax(model.predict(X), axis=1)
    accuracy = accuracy_score(y, y_pred)
    return accuracy

# Load and preprocess test data function
def load_and_preprocess_test_data(test_data_path):
    test_data = pd.read_csv(test_data_path)
    X_test = test_data.drop(columns=['Class', 'name']).values
    y_test = test_data['Class'].values
    scaler = StandardScaler()
    X_test_scaled = scaler.fit_transform(X_test)
    X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))
    return X_test_scaled, y_test

# Function to evaluate model on test data
def evaluate_model(model, X_test, y_test):
    y_pred = (model.predict(X_test) > 0.5).astype(int)
    print("Model Classification Report for Testing Set:")
    print(classification_report(y_test, y_pred))
    accuracy_test = accuracy_score(y_test, y_pred)
    print(f"\nAccuracy for Testing Set: {accuracy_test:.4f}")

# Define EWC loss function
def ewc_loss(labels, preds, model, F, theta_A):
    loss_b = model.loss(labels, preds)
    penalty = compute_elastic_penalty(F, model.trainable_variables, theta_A)
    return loss_b + penalty

# Define function to compute precision matrices
def compute_precision_matrices(model, task_set, num_batches=1, batch_size=32):
    task_set = task_set.repeat()
    precision_matrices = {n: tf.zeros_like(p.value()) for n, p in enumerate(model.trainable_variables)}

    for i, (imgs, labels) in enumerate(task_set.take(num_batches)):
        with tf.GradientTape() as tape:
            preds = model(imgs)
            ll = tf.nn.log_softmax(preds)
        ll_grads = tape.gradient(ll, model.trainable_variables)
        for i, g in enumerate(ll_grads):
            precision_matrices[i] += tf.math.reduce_mean(g ** 2, axis=0) / num_batches

    return precision_matrices

# Define function to compute elastic penalty
def compute_elastic_penalty(F, theta, theta_A, alpha=25):
    penalty = 0
    for i, theta_i in enumerate(theta):
        _penalty = tf.math.reduce_sum(F[i] * (theta_i - theta_A[i]) ** 2)
        penalty += _penalty
    return 0.5 * alpha * penalty

# Define function to train with EWC
def train_with_ewc(model, task_A_set, task_B_set, task_A_test, task_B_test, epochs=3):
    # First we're going to fit to task A and retain a copy of parameters trained on Task A
    model.fit(task_A_set, epochs=epochs)
    theta_A = {n: p.value() for n, p in enumerate(model.trainable_variables.copy())}
    # We'll only compute Fisher once, you can do it whenever
    F = compute_precision_matrices(model, task_A_set, num_batches=1000)

    print("Task A accuracy after training on Task A: {}".format(evaluate(model, task_A_test)))

    # Now we set up the training loop for task B with EWC
    accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
    loss = tf.keras.metrics.SparseCategoricalCrossentropy('loss')

    for epoch in range(epochs * 3):
        accuracy.reset_states()
        loss.reset_states()

        for batch, (imgs, labels) in enumerate(task_B_set):
            with tf.GradientTape() as tape:
                # Make the predictions
                preds = model(imgs)
                # Compute EWC loss
                total_loss = ewc_loss(labels, preds, model, F, theta_A)
            # Compute the gradients of model's trainable parameters wrt total loss
            grads = tape.gradient(total_loss, model.trainable_variables)
            # Update the model with gradients
            model.optimizer.apply_gradients(zip(grads, model.trainable_variables))
            # Report updated loss and accuracy
            accuracy.update_state(labels, preds)
            loss.update_state(labels, preds)
            print("\rEpoch: {}, Batch: {}, Loss: {:.3f}, Accuracy: {:.3f}".format(
                epoch + 1, batch + 1, loss.result().numpy(), accuracy.result().numpy()), flush=True, end=''
            )
        print("")

    print("Task B accuracy after training trained model on Task B: {}".format(evaluate(model, task_B_test)))
    print("Task A accuracy after training trained model on Task B: {}".format(evaluate(model, task_A_test)))

# Load the saved model
model_save_path = '/content/drive/MyDrive/Depression/Model/latest_CNN.h5'
loaded_model = load_model(model_save_path)
print("Model loaded successfully.")

# Evaluate on first test dataset
test_data_path_1 = '/content/drive/MyDrive/Depression/EDAIC Dataset/testing_labels.csv'
X_test_1, y_test_1 = load_and_preprocess_test_data(test_data_path_1)
evaluate_model(loaded_model, X_test_1, y_test_1)

# Evaluate on second test dataset
test_data_path_2 = '/content/drive/MyDrive/Depression/Android Dataset/Interview_test.csv'
X_test_2, y_test_2 = load_and_preprocess_test_data(test_data_path_2)
evaluate_model(loaded_model, X_test_2, y_test_2)

# Fine-tuning the model on Interview dataset
interview_train_path = '/content/drive/MyDrive/Depression/Android Dataset/Interview_train.csv'
interview_train_data = pd.read_csv(interview_train_path)
X_interview = interview_train_data.drop(columns=['name', 'Class']).values
y_interview = interview_train_data['Class'].values

scaler = StandardScaler()
X_interview_scaled = scaler.fit_transform(X_interview)

# Reshape data for CNN
X_interview_scaled_reshaped = X_interview_scaled.reshape((X_interview_scaled.shape[0], X_interview_scaled.shape[1], 1))

# Compile model with smaller learning rate
loaded_model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Use early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model on Interview dataset
loaded_model.fit(X_interview_scaled_reshaped, y_interview, epochs=10, validation_split=0.2, callbacks=[early_stopping])

# Save the fine-tuned model
loaded_model.save('/content/drive/MyDrive/Depression/Model/finetuned_model.h5')

# Evaluate the fine-tuned model on test datasets
evaluate_model(loaded_model, X_test_1, y_test_1)
evaluate_model(loaded_model, X_test_2, y_test_2)

# Define EWC model
ewc_model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(5)
])

ewc_model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics='accuracy')

# Call train_with_ewc function with appropriate datasets
# train_with_ewc(ewc_model, task_A_train, task_B_train, task_A_test, task_B_test)


Model loaded successfully.
77/77 [==============================] - 0s 3ms/step
Model Classification Report for Testing Set:
              precision    recall  f1-score   support

           0       0.78      0.95      0.86      1897
           1       0.26      0.06      0.10       549

    accuracy                           0.75      2446
   macro avg       0.52      0.51      0.48      2446
weighted avg       0.66      0.75      0.69      2446


Accuracy for Testing Set: 0.7522
47/47 [==============================] - 0s 4ms/step
Model Classification Report for Testing Set:
              precision    recall  f1-score   support

           0       0.42      0.94      0.58       624
           1       0.62      0.07      0.13       875

    accuracy                           0.43      1499
   macro avg       0.52      0.51      0.35      1499
weighted avg       0.54      0.43      0.31      1499


Accuracy for Testing Set: 0.4323
Epoch 1/10
85/85 [==============================] - 6s 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


77/77 [==============================] - 0s 3ms/step
Model Classification Report for Testing Set:
              precision    recall  f1-score   support

           0       0.82      0.62      0.71      1897
           1       0.29      0.52      0.37       549

    accuracy                           0.60      2446
   macro avg       0.55      0.57      0.54      2446
weighted avg       0.70      0.60      0.63      2446


Accuracy for Testing Set: 0.5998
47/47 [==============================] - 0s 2ms/step
Model Classification Report for Testing Set:
              precision    recall  f1-score   support

           0       0.60      0.81      0.69       624
           1       0.82      0.61      0.70       875

    accuracy                           0.70      1499
   macro avg       0.71      0.71      0.70      1499
weighted avg       0.73      0.70      0.70      1499


Accuracy for Testing Set: 0.6958


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the evaluate function
def evaluate(model, X, y):
    y_pred = np.argmax(model.predict(X), axis=1)
    accuracy = accuracy_score(y, y_pred)
    return accuracy

# Load and preprocess test data function
def load_and_preprocess_test_data(test_data_path):
    test_data = pd.read_csv(test_data_path)
    X_test = test_data.drop(columns=['Class', 'name']).values
    y_test = test_data['Class'].values
    scaler = StandardScaler()
    X_test_scaled = scaler.fit_transform(X_test)
    X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))
    return X_test_scaled, y_test

# Function to evaluate model on test data
def evaluate_model(model, X_test, y_test):
    y_pred = (model.predict(X_test) > 0.5).astype(int)
    print("Model Classification Report for Testing Set:")
    print(classification_report(y_test, y_pred))
    accuracy_test = accuracy_score(y_test, y_pred)
    print(f"\nAccuracy for Testing Set: {accuracy_test:.4f}")

# Define L2 penalty function
def l2_penalty(theta, theta_A):
    penalty = 0
    for i, theta_i in enumerate(theta):
        _penalty = tf.math.reduce_sum((theta_i - theta_A[i]) ** 2)
        penalty += _penalty
    return 0.5 * penalty

# Define function to train with L2 regularization
def train_with_l2(model, task_A_train, task_B_train, X_test_A, y_test_A, X_test_B, y_test_B, epochs=6):
    model.fit(task_A_train, epochs=epochs)
    theta_A = {n: p.value() for n, p in enumerate(model.trainable_variables.copy())}

    print("Task A accuracy after training on Task A: {}".format(evaluate(model, X_test_A, y_test_A)))

    accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
    loss = tf.keras.metrics.SparseCategoricalCrossentropy('loss')

    for epoch in range(epochs):
        accuracy.reset_states()
        loss.reset_states()
        for batch, (imgs, labels) in enumerate(task_B_train):
            with tf.GradientTape() as tape:
                preds = model(imgs)
                total_loss = model.loss(labels, preds) + l2_penalty(model.trainable_variables, theta_A)
            grads = tape.gradient(total_loss, model.trainable_variables)
            model.optimizer.apply_gradients(zip(grads, model.trainable_variables))

            accuracy.update_state(labels, preds)
            loss.update_state(labels, preds)
            print("\rEpoch: {}, Batch: {}, Loss: {:.3f}, Accuracy: {:.3f} - Training on Task B".format(
                epoch + 1, batch + 1, loss.result().numpy(), accuracy.result().numpy()), flush=True, end=''
            )
        print("")

    print("Task B accuracy after training trained model on Task B: {}".format(evaluate(model, X_test_B, y_test_B)))
    print("Task A accuracy after training trained model on Task B: {}".format(evaluate(model, X_test_A, y_test_A)))

# Define function to compute precision matrices
def compute_precision_matrices(model, task_set, num_batches=1, batch_size=32):
    task_set = task_set.repeat()
    precision_matrices = {n: tf.zeros_like(p.value()) for n, p in enumerate(model.trainable_variables)}

    for i, (imgs, labels) in enumerate(task_set.take(num_batches)):
        with tf.GradientTape() as tape:
            preds = model(imgs)
            ll = tf.nn.log_softmax(preds)
        ll_grads = tape.gradient(ll, model.trainable_variables)
        for i, g in enumerate(ll_grads):
            precision_matrices[i] += tf.math.reduce_mean(g ** 2, axis=0) / num_batches

    return precision_matrices

# Define function to compute elastic penalty
def compute_elastic_penalty(F, theta, theta_A, alpha=25):
    penalty = 0
    for i, theta_i in enumerate(theta):
        _penalty = tf.math.reduce_sum(F[i] * (theta_i - theta_A[i]) ** 2)
        penalty += _penalty
    return 0.5 * alpha * penalty

# Define EWC loss function
def ewc_loss(labels, preds, model, F, theta_A):
    loss_b = model.loss(labels, preds)
    penalty = compute_elastic_penalty(F, model.trainable_variables, theta_A)
    return loss_b + penalty

# Define function to train with EWC
def train_with_ewc(model, task_A_set, task_B_set, X_test_A, y_test_A, X_test_B, y_test_B, epochs=3):
    # First we're going to fit to task A and retain a copy of parameters trained on Task A
    model.fit(task_A_set, epochs=epochs)
    theta_A = {n: p.value() for n, p in enumerate(model.trainable_variables.copy())}
    # We'll only compute Fisher once, you can do it whenever
    F = compute_precision_matrices(model, task_A_set, num_batches=1000)

    print("Task A accuracy after training on Task A: {}".format(evaluate(model, X_test_A, y_test_A)))

    # Now we set up the training loop for task B with EWC
    accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
    loss = tf.keras.metrics.SparseCategoricalCrossentropy('loss')

    for epoch in range(epochs * 3):
        accuracy.reset_states()
        loss.reset_states()

        for batch, (imgs, labels) in enumerate(task_B_set):
            with tf.GradientTape() as tape:
                # Make the predictions
                preds = model(imgs)
                # Compute EWC loss
                total_loss = ewc_loss(labels, preds, model, F, theta_A)
            # Compute the gradients of model's trainable parameters wrt total loss
            grads = tape.gradient(total_loss, model.trainable_variables)
            # Update the model with gradients
            model.optimizer.apply_gradients(zip(grads, model.trainable_variables))
            # Report updated loss and accuracy
            accuracy.update_state(labels, preds)
            loss.update_state(labels, preds)
            print("\rEpoch: {}, Batch: {}, Loss: {:.3f}, Accuracy: {:.3f} - Training on Task B".format(
                epoch + 1, batch + 1, loss.result().numpy(), accuracy.result().numpy()), flush=True, end=''
            )
        print("")

    print("Task B accuracy after training trained model on Task B: {}".format(evaluate(model, X_test_B, y_test_B)))
    print("Task A accuracy after training trained model on Task B: {}".format(evaluate(model, X_test_A, y_test_A)))

# Load the saved model
model_save_path = '/content/drive/MyDrive/Depression/Model/latest_CNN.h5'
loaded_model = load_model(model_save_path)
print("Model loaded successfully.")

# Evaluate on first test dataset
test_data_path_1 = '/content/drive/MyDrive/Depression/EDAIC Dataset/testing_labels.csv'
X_test_1, y_test_1 = load_and_preprocess_test_data(test_data_path_1)
evaluate_model(loaded_model, X_test_1, y_test_1)

# Evaluate on second test dataset
test_data_path_2 = '/content/drive/MyDrive/Depression/Android Dataset/Interview_test.csv'
X_test_2, y_test_2 = load_and_preprocess_test_data(test_data_path_2)
evaluate_model(loaded_model, X_test_2, y_test_2)

# Fine-tuning the model on Interview dataset
interview_train_path = '/content/drive/MyDrive/Depression/Android Dataset/Interview_train.csv'
interview_train_data = pd.read_csv(interview_train_path)
X_interview = interview_train_data.drop(columns=['name', 'Class']).values
y_interview = interview_train_data['Class'].values

scaler = StandardScaler()
X_interview_scaled = scaler.fit_transform(X_interview)

# Reshape data for CNN
X_interview_scaled_reshaped = X_interview_scaled.reshape((X_interview_scaled.shape[0], X_interview_scaled.shape[1], 1))

# Compile model with smaller learning rate
loaded_model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Use early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model on Interview dataset
loaded_model.fit(X_interview_scaled_reshaped, y_interview, epochs=10, validation_split=0.2, callbacks=[early_stopping])

# Save the fine-tuned model
loaded_model.save('/content/drive/MyDrive/Depression/Model/finetuned_model.h5')

# Evaluate the fine-tuned model on test datasets
evaluate_model(loaded_model, X_test_1, y_test_1)
evaluate_model(loaded_model, X_test_2, y_test_2)

# Load dataset A
task_A_train_path = '/content/drive/MyDrive/Depression/EDAIC Dataset/training_labels.csv'
task_A_train_data = pd.read_csv(task_A_train_path)

# Load dataset B
task_B_train_path = '/content/drive/MyDrive/Depression/Android Dataset/Interview_train.csv'
task_B_train_data = pd.read_csv(task_B_train_path)

# Call train_with_l2 function with appropriate datasets
train_with_l2(l2_model, task_A_train_data, task_B_train_data, X_test_1, y_test_1, X_test_2, y_test_2)

# Call train_with_ewc function with appropriate datasets
train_with_ewc(ewc_model, task_A_train_data, task_B_train_data, X_test_1, y_test_1, X_test_2, y_test_2)


Model loaded successfully.
77/77 [==============================] - 0s 3ms/step
Model Classification Report for Testing Set:
              precision    recall  f1-score   support

           0       0.78      0.95      0.86      1897
           1       0.26      0.06      0.10       549

    accuracy                           0.75      2446
   macro avg       0.52      0.51      0.48      2446
weighted avg       0.66      0.75      0.69      2446


Accuracy for Testing Set: 0.7522
47/47 [==============================] - 0s 3ms/step
Model Classification Report for Testing Set:
              precision    recall  f1-score   support

           0       0.42      0.94      0.58       624
           1       0.62      0.07      0.13       875

    accuracy                           0.43      1499
   macro avg       0.52      0.51      0.35      1499
weighted avg       0.54      0.43      0.31      1499


Accuracy for Testing Set: 0.4323
Epoch 1/10
85/85 [==============================] - 3s 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


77/77 [==============================] - 0s 2ms/step
Model Classification Report for Testing Set:
              precision    recall  f1-score   support

           0       0.81      0.65      0.72      1897
           1       0.27      0.46      0.34       549

    accuracy                           0.61      2446
   macro avg       0.54      0.55      0.53      2446
weighted avg       0.69      0.61      0.64      2446


Accuracy for Testing Set: 0.6075
47/47 [==============================] - 0s 3ms/step
Model Classification Report for Testing Set:
              precision    recall  f1-score   support

           0       0.59      0.85      0.70       624
           1       0.84      0.58      0.69       875

    accuracy                           0.69      1499
   macro avg       0.72      0.71      0.69      1499
weighted avg       0.74      0.69      0.69      1499


Accuracy for Testing Set: 0.6918


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).